<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toroton City</font></h1>

## Introduction

In this notebook, we will scrape data from a wikepedia page 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

that has information about Toronto neighborhood to provide a dataframe for segmentation later.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Scrape Data and Explore Dataset

#### Load and explore the data

Next, let's scrape the data using BeautifulSoup package

In [2]:
from bs4 import BeautifulSoup as BS
#!conda install -c conda-forge Requests --yes

In [3]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
results.status_code

200

In [4]:
#assign data to Beautifulsoup for Scraping
c = results.content
soup = BS(c)

In [5]:
#get the column names
dataFrame = pd
column_names = []
for n in soup.table.find_all('th'):
    
    column_names.append((n.text.rstrip()))   
column_names #data = map(lambda soup.table: (soup.table.find_all(text=True),souptable.find_all('tr')))

['Postcode', 'Borough', 'Neighbourhood']

In [6]:
column_names[-1].replace(r'\n',' ')
neighborhoods = pd.DataFrame(columns=column_names)# instantiate the dataframe
neighborhoods

,Postcode,Borough,Neighbourhood


Reading through the table and assign data to 

In [7]:
for i, row in enumerate(soup.table.findAll('tr')):
    rowValue = []
    for a in row.findAll('td'):
        rowValue.append(a.text)
    if len(rowValue) == 3:
        neighborhoods.loc[i-1] = rowValue

In [8]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [9]:
#removing '\n'
neighborhoods = neighborhoods.replace('\n',' ', regex=True)
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
neighborhoods.shape

(289, 3)

### Cleaning up the data 

Removing all the 'not assigned' row

In [11]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [12]:
cleanNbh = neighborhoods.reset_index()

In [13]:
del cleanNbh['index'] #make neighborhood name same as Borough if value is "not assigned"

In [14]:
cleanNbh.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Changing 'not assigned" to the Borough name

In [15]:
cleanNbh.Neighbourhood = cleanNbh.Neighbourhood.replace('Not assigned ',cleanNbh.Borough)

In [16]:
cleanNbh.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


Using groupby() method to combine all neighborhoods in the same postal code together. 

In [17]:
cleanNbh = pd.DataFrame(cleanNbh.groupby(['Postcode', 'Borough'], as_index=False, sort=False)['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)))

In [18]:
cleanNbh.head(15)

,,0
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront , Regent Park"
M6A,North York,"Lawrence Heights , Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge , Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens , Parkview Hill"


Reset index to allow the dataframe to have three columns

In [19]:
cleanNbh = cleanNbh.reset_index()
cleanNbh.head()

,Postcode,Borough,0
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [20]:
cleanNbh.columns = ['Postcode', 'Borough', 'Neighbourhood']
cleanNbh.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge , Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens , Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson , Garden District"


Printing out the final shape of the data

In [21]:
cleanNbh.shape

(103, 3)

## Getting the longtitude and Latitude of each Postal Code

Read geosptial data

In [22]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.columns = ['Postcode', 'Latitude', 'Longitude']

In [23]:
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge coordinate with the clean neighorhood dataframe using postcode columns

In [24]:
nbhWithCoor = pd.merge(cleanNbh, coordinates, on = 'Postcode', how = 'left')

In [25]:
nbhWithCoor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [26]:
nbhWithCoor.shape

(103, 5)

## Exploring Toronto and Neighborhoods

### Create a map of Toronto with Folium and add neighborhoods on top

In [27]:
latitude = 43.654260;
longitude = -79.36063
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbhWithCoor['Latitude'], nbhWithCoor['Longitude'], nbhWithCoor['Borough'], nbhWithCoor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursqaure Credentials and Version

In [28]:
LIMIT = 100
radius = 500
CLIENT_ID = 'ERVR3FIDFM1HN22OBNBPE4O1X3TBMR4IXTC5LRM51RLHHJ0G' # your Foursquare ID
CLIENT_SECRET = 'URA5FQSSJQ2W0TNUGAEH1ZOBYSSIAYZEMQOXJA1LKOKPEJ4D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ERVR3FIDFM1HN22OBNBPE4O1X3TBMR4IXTC5LRM51RLHHJ0G
CLIENT_SECRET:URA5FQSSJQ2W0TNUGAEH1ZOBYSSIAYZEMQOXJA1LKOKPEJ4D


### Exploring Downtown Toronto

First, get the latitude and longitude of the neighborhood

In [29]:
toronto_data = nbhWithCoor[nbhWithCoor['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson , Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Create functions to get venues for neighborhoods

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront , Regent Park 
Ryerson , Garden District 
St. James Town 
Berczy Park 
Central Bay Street 
Christie 
Adelaide , King , Richmond 
Harbourfront East , Toronto Islands , Union Station 
Design Exchange , Toronto Dominion Centre 
Commerce Court , Victoria Hotel 
Harbord , University of Toronto 
Chinatown , Grange Park , Kensington Market 
CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara 
Rosedale 
Stn A PO Boxes 25 The Esplanade 
Cabbagetown , St. James Town 
First Canadian Place , Underground city 
Church and Wellesley 


In [32]:
print(toronto_venues.shape)

(1283, 7)


Counting number of venues for different neighborhoods

In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide , King , Richmond",100,100,100,100,100,100
Berczy Park,53,53,53,53,53,53
"CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara",14,14,14,14,14,14
"Cabbagetown , St. James Town",48,48,48,48,48,48
Central Bay Street,82,82,82,82,82,82
"Chinatown , Grange Park , Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court , Victoria Hotel",100,100,100,100,100,100


In [34]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront , Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront , Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront , Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront , Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront , Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


## Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot= toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront , Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront , Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
toronto_onehot.shape

(1283, 204)

Group rows by neighborhood and calculating mean of occurrence

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide , King , Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00000,0.00,0.00,0.010000,0.010000,0.000000,0.02,0.0000,0.00,0.0000,0.000000,0.020000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.00,0.000000,0.00,0.020000,0.010000,0.000000,0.060000,0.000000,0.000000,0.000000,0.00,0.00000,0.020000,0.010000,0.070000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.020000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.00000,0.000000,0.000000,0.00,0.010000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00000,0.030000,0.00,0.010000,0.010000,0.00,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.0

In [38]:
toronto_grouped.shape

(18, 204)

Print each neighborhoo along with top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond ----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park ----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.06
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4      Farmers Market  0.04


----CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara ----
              venue  freq
0  Airport Terminal  0.14
1   Airport Service  0.14
2    Airport Lounge  0.14
3     Boat or Ferry  0.07
4      Airport Gate  0.07


----Cabbagetown , St. James Town ----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.08
2   Indian Restaurant  0.04
3  Italian Restaurant  0.04
4                Café  0.04


----Central Bay Street ----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.

## putting the above information into a dataframe

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Restaurant,Hotel,Bar,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop,Café,Steakhouse,Restaurant
2,"CN Tower , Bathurst Quay , Island airport , Ha...",Airport Terminal,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane,Boutique
3,"Cabbagetown , St. James Town",Coffee Shop,Restaurant,Café,Pub,Bakery,Pizza Place,Park,Chinese Restaurant,Indian Restaurant,Italian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Salad Place
5,"Chinatown , Grange Park , Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Noodle House
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Convenience Store,Restaurant,Diner,Baby Store,Italian Restaurant
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Burger Joint,Gay Bar,Restaurant,Bubble Tea Shop,Café,Men's Store,Mediterranean Restaurant
8,"Commerce Court , Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Seafood Restaurant
9,"Design Exchange , Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Gym,Sports Bar,Deli / Bodega,Italian Restaurant


## Clustering neighborhoods

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 3, 4, 0, 0, 0], dtype=int32)

In [43]:
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson , Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [44]:
toronto_data.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
1,M5B,Downtown Toronto,"Ryerson , Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Ramen Restaurant,Restaurant,Movie Theater,Sandwich Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cocktail Bar,Gastropub,Bakery,Italian Restaurant,Cosmetics Shop
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop,Café,Steakhouse,Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Salad Place


Create maps with cluster

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront , Regent Park",0,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
1,"Ryerson , Garden District",0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Ramen Restaurant,Restaurant,Movie Theater,Sandwich Place
3,Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop,Café,Steakhouse,Restaurant
4,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Salad Place
7,"Harbourfront East , Toronto Islands , Union St...",0,Coffee Shop,Hotel,Pizza Place,Aquarium,Café,Sports Bar,Italian Restaurant,Scenic Lookout,Brewery,Steakhouse
8,"Design Exchange , Toronto Dominion Centre",0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Gym,Sports Bar,Deli / Bodega,Italian Restaurant
9,"Commerce Court , Victoria Hotel",0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Seafood Restaurant
10,"Harbord , University of Toronto",0,Café,Coffee Shop,Theater,Bar,Japanese Restaurant,Restaurant,Bakery,Bookstore,Pub,Poutine Place
12,"CN Tower , Bathurst Quay , Island airport , Ha...",0,Airport Terminal,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane,Boutique
13,Rosedale,0,Park,Trail,Playground,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Stn A PO Boxes 25 The Esplanade,1,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Cocktail Bar,Beer Bar,Creperie,Farmers Market,Pub


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,2,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cocktail Bar,Gastropub,Bakery,Italian Restaurant,Cosmetics Shop


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,3,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Convenience Store,Restaurant,Diner,Baby Store,Italian Restaurant
11,"Chinatown , Grange Park , Kensington Market",3,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Noodle House


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Adelaide , King , Richmond",4,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Restaurant,Hotel,Bar,Gym
